# Playing with TF, following examples on the web
Mostly from the project tensorflow_models

### A namedtuple can be used for storing hyper-params
It generates a class which its instances can return tuples, but fields can be referred by name. However, it would not work well with take_from_struct style params. So it would better be using a dictionary and fetching default values if a param doesnt exists.

In [12]:
from collections import namedtuple
HyperParams = namedtuple('HyperParams', 'lr, w_decay')
hp = HyperParams(lr=0.1, w_decay=1e-3)
print hp
a,b=hp
print a,b
print hp.lr

HyperParams(lr=0.1, w_decay=0.001)
0.1 0.001
0.1


## Threads and queues
[link](https://www.tensorflow.org/versions/r0.10/how_tos/threading_and_queues/index.html)
A **queue is a node** in a TensorFlow graph. It's a stateful node, like a variable: other nodes can modify its content. In particular, nodes can enqueue new items in to the queue, or dequeue existing items from the queue.

A simple example: A "first in, first out" queue (FIFOQueue) init it with zeros. Then we'll construct a graph that takes an item off the queue, adds one to that item, and puts it back on the end of the queue. Slowly, the numbers on the queue increase.

![](https://www.tensorflow.org/versions/r0.10/images/IncremeterFifoQueue.gif)

### Coordinator
[link](https://www.tensorflow.org/versions/r0.10/how_tos/threading_and_queues/index.html#Coordinator)

### QueueRunner

The QueueRunner class creates a number of threads that repeatedly run an enqueue op. These threads can use a coordinator to stop together. In addition, a queue runner runs a closer thread that automatically closes the queue if an exception is reported to the coordinator.

**Implementation:** First **build a graph** with **a Queue for input examples**. Add **ops** that **process examples** and **enqueue them** in the queue. Add **training ops** that start by **dequeueing** from the queue

    example = ...ops to create one example...
    # Create a queue, and an op that enqueues examples one at a time in the queue.
    queue = tf.RandomShuffleQueue(...)
    enqueue_op = queue.enqueue(example)
    # Create a training graph that starts by dequeuing a batch of examples.
    inputs = queue.dequeue_many(batch_size)
    train_op = ...use 'inputs' to build the training part of the graph...


In the Python **training program**, create a **QueueRunner** that will **run threads** to process and enqueue examples. Create a Coordinator and **ask the queue runner to start its threads with the coordinator**. Write a training loop that also uses the coordinator.

    # Create a queue runner that will run 4 threads in parallel to enqueue
    # examples.
    qr = tf.train.QueueRunner(queue, [enqueue_op] * 4)

    # Launch the graph.
    sess = tf.Session()
    # Create a coordinator, launch the queue runner threads.
    coord = tf.train.Coordinator()
    enqueue_threads = qr.create_threads(sess, coord=coord, start=True)
    # Run the training loop, controlling termination with the coordinator.
    for step in xrange(1000000):
        if coord.should_stop():
            break
        sess.run(train_op)
    # When done, ask the threads to stop.
    coord.request_stop()
    # And wait for them to actually do it.
    coord.join(enqueue_threads)




## Reader and queues  for input pipeline


The TF pipeline supports reading data from multiple files (possibly with multiple threads), preprocessing it on the fly and feedining it as batches for training. This pipeline involves 3 queues that are piped.

#### Here is an illustration
![](https://www.tensorflow.org/versions/r0.10/images/AnimatedFileQueues.gif)

#### Here is a TL;DR from the TF (`QueueRunner`) tutorial
One set of threads generates filenames in a queue; a second set of threads read records from the files, processes them, and enqueues tensors on a second queue; a third set of threads (not shown in animation) dequeues these input records to construct batches and runs them through training operations.

#### In more details
Many of the `tf.train` functions listed above add `QueueRunner` objects to your graph. These require that you call `tf.train.start_queue_runners` before running any training or inference steps, or it will hang forever. This will start threads that run the input pipeline, filling the example queue so that the dequeue to get the examples will succeed. This is best combined with a `tf.train.Coordinator` to cleanly shut down these threads when there are errors. If you set a limit on the number of epochs, that will use an epoch counter that will need to be initialized. 